### Import necessary libraries and packages



In [ ]:
pip install -U transformers accelerate bitsandbytes mistral-common accelerate mistral_inference  -qq

#### Get file from LLM zoomcamp course repo

In [46]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-07-07 08:58:53--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-07 08:58:53 (49.4 MB/s) - 'minsearch.py' saved [3832/3832]



#### Get json file from course repo and create TF-IDF

In [47]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

#### search query based on cosine-similarity

In [48]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig,  pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-07 08:28:37.720185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 08:28:37.720296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 08:28:37.917311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

#### Mistral models can be downloaded with huggingface authenticatiion

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

In [4]:
from huggingface_hub import login
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Get Mistral-7B-Instruct-v0.1 from huggingface

In [18]:
 
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             quantization_config=quantization_config,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Create pipeline

In [158]:
generator = pipeline("text-generation", model=model, 
                     tokenizer=tokenizer
                    )

#### Build prompt

In [162]:
def build_prompt(query, search_results):
    prompt_template = [
 {"role": "system", "content": "Your response should be clear and concise"},
 

    ]

   
    
    for doc in search_results:
        prompt_template.append({'role':'user','content':doc['question']})
        prompt_template.append({'role':'assistant','content':doc['text']})
        
    prompt_template.append({'role':'user','content':query})
    prompt = tokenizer.apply_chat_template(prompt_template, tokenize=False,
            add_generation_prompt=True)
    return prompt

def llm(prompt,temp):
    config=config={'max_length':1000,
                'temperature':temp,
                'do_sample':True,
                'num_return_sequences':1,
               # 'num_beams':2,         
                'top_p':0.95,         
                #'top_k':50,        
                'eos_token_id':tokenizer.eos_token_id,
                'pad_token_id':tokenizer.eos_token_id,
               }
    response = generator(prompt,**config)
    response_final = response[0]['generated_text'].split('[/INST]')[-1].strip()
    return response_final

#### Get results and create prompt from it and generate result using llm

In [160]:
def rag(query,temp=0.1):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt,temp)
    return answer

#### The search based on the json document data

#### Temperature 0.1

In [169]:
%%time
print(rag("I just discovered the course. Can I still enroll it?"))

Yes, you can still enroll the course. The course is open for enrollment until the start date.
You can check the course details and register using the link provided.
Please note that the course will start on 15th Jan 2024 at 17h00.
CPU times: user 6.59 s, sys: 490 ms, total: 7.08 s
Wall time: 7.08 s


#### Let's try using different temperature values and examine the differences in the results.

In [148]:
import numpy as np
from IPython.display import display, Markdown

In [167]:
%%time
for temp in np.round(np.linspace(0.2,1.0,9),1):
    display(Markdown(f"### Temperature: {temp}"))
    result=rag('I just discovered the course. Can I still enroll it?',temp)
    display(Markdown(f"Result: {result}"))
    print("-"*100)

### Temperature: 0.2

Result: Yes, you can still enroll in the course. The course will start on 15th Jan 2024 at 17h00.
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

----------------------------------------------------------------------------------------------------


### Temperature: 0.3

Result: Yes, you can still enroll in the course even if it has already started.
You can check the course schedule and see if there are any upcoming live sessions or office hours that you can attend.
You can also check the course materials and start working on the homeworks at your own pace.
Please note that the course will have deadlines for turning in the final projects, so it's best to start working on them as soon as possible.

...

Result: Yes, you can enroll in this course even if it has already started.
Keep in mind that you might miss some of the early parts of the course, but you can still catch up by watching the recorded live sessions, homework solutions, and reading the course materials.
It's always better to join a course as early as possible so you have time to catch up and keep up with the pace.

----------------------------------------------------------------------------------------------------


### Temperature: 1.0

Result: The course has started on 15th Jan 2024 at 17h00, so it is too late to enroll now.
However, you can check if there will be another cohort of this course in the future and enroll as soon as it starts. In any case, please keep in mind that the course is designed to be completed in a specific sequence of dates, and if you missed the start date, some of the assignments might be inaccessible.

----------------------------------------------------------------------------------------------------
CPU times: user 1min 12s, sys: 4.04 s, total: 1min 16s
Wall time: 1min 16s
